In [2]:
import mne                                                              # Librería de python para explorar, visualizar,
mne.set_log_level('WARNING')                                            # y analizar datos neurofisiológicos humanos.
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb                                                    
color = ['green', 'blue','red','cyan', 'magenta', 'yellow','k','w']     # Paleta de colores para diferenciar las ondas

import a_funciones as a_fun                                             # Funciones Alex

xlabel = 'Muestra'                                                      # Abscisas
ylabel = 'Amplitud (uV)'                                                # Ordenadas

In [13]:
# In[1]:
#import mne
#mne.set_log_level('WARNING')

#import scipy.io
#import numpy as np

# First load the template.  This is the signal that will be used to DRUG the basal EEG stream.
mat = scipy.io.loadmat('./dataset/ERPTemplate.mat')

routput = mat['routput']

# In this ERPTemplate, there are two different template signals that are good.
erptemplate1 = routput[0][7][0][1][0][0][0][7] 
erptemplate2 = routput[0][7][0][1][0][0][0][0] 

# The original ERPTemplate dataset has a sampling frequency of 256 so I need to perform a small downsampling to 250 Hz
erptemplate1 = np.delete( erptemplate1, range(0,256,43),0)
erptemplate2 = np.delete( erptemplate2, range(0,256,43),0)

# Use this for testing  (get a ZERO signal)
#erptemplate1 = np.zeros((250,8))

# Randomize amplitude and jitter.
# Find the right locations where this should be inserted in the stream.
# Insert the signal mantaining the continiuity of the EEG.
def DrugSignal(signal, t_flash):
    '''
    Randomize amplitude and jitter
    Find the right locations where this should be inserted in the stream
    Insert the template mantaining the continuity and physiological meaning of the EEG
    '''
    for i in range(0,4200):
        if (t_flash[i,3]==2):
            signal[t_flash[i,0]-1:t_flash[i,0]+250-1,:] += (erptemplate1*3)

    return signal

# Now load the basal EEG stream
mat = scipy.io.loadmat('./dataset/p300-subject-25.mat')
#mat = scipy.io.loadmat('./dataset/p300-subject-26.mat')
#mat = scipy.io.loadmat('/Users/rramele/./GoogleDrive/Data/P300/p300-subject-21.mat')
#mat = scipy.io.loadmat('/Users/rramele/./GoogleDrive/Data/P300/p300-subject-06.mat')

# In[1]:

# coding: latin-1
# Data point zero for the eight channels.  Should be in V.
signal = mat['data'][0][0][0] 
#* pow(10,6)

# Trials
t_trials = mat['data'][0][0][3]

# Flash matrix
t_flash = mat['data'][0][0][4]

signal = DrugSignal(signal, t_flash)

t_stim = mat['data'][0][0][2]
t_type = mat['data'][0][0][1]

# ch_names -> Los nombres de los canales
ch_names=[ 'Fz'  ,  'Cz',    'P3' ,   'Pz'  ,  'P4'  ,  'PO7'   , 'PO8'   , 'Oz']
# ch_types -> Un vector de tamaño 8 con el'eeg' repetido
ch_types= ['eeg'] * signal.shape[1]
# ch_names_events -> Le agrega  't_stim' y't_type' 
ch_names_events = ch_names + ['t_stim']+ ['t_type']
# ch_types_events -> le agrega dos 'misc' 
ch_types_events = ch_types + ['misc'] + ['misc']
# signal_events -> Se agrega a la señal drogada 't_stim' y 't_type'
signal_events = np.concatenate([signal, t_stim, t_type],1)
df_signal_events = a_fun.to_df(signal_events)
# Acá crea los eventos. No está muy claro. Revisar.
info_events = mne.create_info(ch_names_events,250, ch_types_events)
# eeg es el mismo info_events pero con Duration	00:23:53 (HH:MM:SS)
eeg = mne.io.RawArray(signal_events.T, info_events)
#df_eeg = a_fun.to_df(eeg)  

# Do some basic signal processing (1-20 band pass filter)
#fig=eeg.plot_psd()
#eeg.filter(1,20)
#fig=eeg.plot_psd()
event_times = mne.find_events(eeg, stim_channel='t_type')  
df_event_times = a_fun.to_df(event_times)  

#plt.figure(figsize=(30,8))
#axes = plt.gca()
#plt.plot(df_t_flash['sample'], df_t_flash[3])
#axes.set_title('Señal t_flash[3] completa'), axes.title.set_size(30)
#axes.set_xlabel(xlabel), axes.set_ylabel(ylabel)
#axes.xaxis.label.set_size(20), axes.yaxis.label.set_size(20)
#plt.grid(), plt.show()

#plt.figure(figsize=(30,8))
#plt.plot(df_eeg)
#eeg.plot(scalings='auto',
#    n_channels=8,
#    events=event_times,
#    block=True,
#    )   # scalings=10e-05

In [12]:
df_signal_events

,0,1,2,3,4,5,6,7,8,9,sample
0,-25.841171,-10.557603,-6.211982,-15.843910,-13.558990,20.122635,-11.946080,6.265652,0.0,0.0,0
1,-34.338673,-18.944792,-17.603668,-26.940290,-23.392313,-17.507841,-31.759108,-16.000412,0.0,0.0,1
2,-37.689247,-19.064447,-18.308565,-25.747719,-21.724098,-18.072618,-28.811285,-11.629916,0.0,0.0,2
3,-24.009695,-6.639306,-11.457250,-11.021503,-1.366220,-5.843951,-10.881763,3.213532,0.0,0.0,3
4,-14.699684,-0.977608,-3.606690,1.438218,18.299120,10.070486,9.120290,27.182615,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
358395,8.348876,-1.448800,-7.263316,-10.008823,0.042449,-10.695869,-3.517254,-14.230025,0.0,0.0,358395
358396,10.000939,-1.899947,-9.188922,-9.475272,6.371345,-23.403809,3.944809,-13.214551,0.0,0.0,358396
358397,8.231050,-6.893495,-10.014253,-9.674665,6.783853,-25.591261,4.212073,-12.678520,0.0,0.0,358397
358398,6.005248,-10.328425,-9.807632,-10.479656,4.436954,-17.936047,-0.169820,-10.185187,0.0,0.0,358398


In [16]:
signal_events_T = signal_events.T
df_signal_events_T = a_fun.to_df(signal_events_T)
df_signal_events_T

,0,1,2,3,4,5,6,7,8,9,...,358391,358392,358393,358394,358395,358396,358397,358398,358399,sample
0,-25.841171,-34.338673,-37.689247,-24.009695,-14.699684,-6.905956,25.883667,-14.083708,-42.580444,-32.301765,...,21.357687,17.920532,16.698196,16.280077,8.348876,10.000939,8.231050,6.005248,5.879085,0
1,-10.557603,-18.944792,-19.064447,-6.639306,-0.977608,4.759079,36.015572,-4.600820,-24.832602,-10.954214,...,10.089139,10.295307,12.954316,9.467443,-1.448800,-1.899947,-6.893495,-10.328425,-10.314727,1
2,-6.211982,-17.603668,-18.308565,-11.457250,-3.606690,1.665619,28.016043,1.976110,-5.260470,5.330559,...,2.106967,4.256704,11.400371,8.283445,-7.263316,-9.188922,-10.014253,-9.807632,-7.537188,2
3,-15.843910,-26.940290,-25.747719,-11.021503,1.438218,6.454340,32.570457,-0.446925,-11.612387,2.850624,...,2.039568,3.635427,7.153916,4.319250,-10.008823,-9.475272,-9.674665,-10.479656,-8.278395,3
4,-13.558990,-23.392313,-21.724098,-1.366220,18.299120,26.683174,60.801224,20.095476,1.500922,10.216281,...,8.648603,9.021157,14.891686,15.101629,0.042449,6.371345,6.783853,4.436954,5.545664,4
5,20.122635,-17.507841,-18.072618,-5.843951,10.070486,12.585777,29.851191,3.899734,5.543776,8.052427,...,-12.525366,-0.769132,25.264278,20.814129,-10.695869,-23.403809,-25.591261,-17.936047,-3.654313,5
6,-11.946080,-31.759108,-28.811285,-10.881763,9.120290,13.611430,27.427492,11.805900,3.472059,5.542897,...,-0.895235,-0.421269,4.242666,6.595631,-3.517254,3.944809,4.212073,-0.169820,-0.059939,6
7,6.265652,-16.000412,-11.629916,3.213532,27.182615,25.152113,27.701725,8.699254,6.472950,17.444468,...,-12.362241,-7.790727,2.947068,3.572793,-14.230025,-13.214551,-12.678520,-10.185187,-3.752876,7
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9
